# WIP

In [1]:
import h2o
from h2o.automl import H2OAutoML
from sklearn.preprocessing import LabelEncoder
import pandas as pd

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "19" 2022-09-20; OpenJDK Runtime Environment (build 19+36-2238); OpenJDK 64-Bit Server VM (build 19+36-2238, mixed mode, sharing)
  Starting server from /home/shishiriyer/.local/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpk5whb7_v
  JVM stdout: /tmp/tmpk5whb7_v/h2o_shishiriyer_started_from_python.out
  JVM stderr: /tmp/tmpk5whb7_v/h2o_shishiriyer_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,America/Los_Angeles
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.4
H2O_cluster_version_age:,1 month and 6 days
H2O_cluster_name:,H2O_from_python_shishiriyer_k83bnw
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.770 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [3]:
# Load the dataset
train_df = pd.read_csv('archive/train.csv')
test_df = pd.read_csv('archive/test_public.csv')
coord_lookup = pd.read_csv('archive/metaData_taxistandsID_name_GPSlocation.csv')

In [4]:
def get_lon(station_no):
    return float(coord_lookup.loc[coord_lookup['ID'] == station_no]['Longitude'].iloc[0])

def get_lat(station_no):
    return float(coord_lookup.loc[coord_lookup['ID'] == station_no]['Latitude'].iloc[0])

In [5]:
encoder = LabelEncoder()
train_df['CALL_TYPE'] = encoder.fit_transform(train_df['CALL_TYPE'])

# Normalize timestamp
train_df['TIMESTAMP'] = pd.to_datetime(train_df['TIMESTAMP'], unit='s')
train_df['MINUTE'] = train_df['TIMESTAMP'].dt.minute
train_df['HOUR'] = train_df['TIMESTAMP'].dt.hour
train_df['DAY'] = train_df['TIMESTAMP'].dt.dayofweek
train_df['MONTH'] = train_df['TIMESTAMP'].dt.month

# Split the training data based on whether CALL_TYPE == 'B' (i.e. there is an origin stand present)
has_origin = train_df['CALL_TYPE'] == 1
train_df_origin = train_df[has_origin]
train_df_nan = train_df[~has_origin]
train_df_origin = train_df_origin.dropna(subset=['ORIGIN_STAND'])



# Split the data into features and target
features_nan = pd.DataFrame({ 'MINUTE': train_df_nan['MINUTE'], 'HOUR': train_df_nan['HOUR'], 'DAY': train_df_nan['DAY'], 'MONTH': train_df_nan['MONTH'] })
target_nan = train_df_nan['POLYLINE'].str.count(',') * 15


target_origin = train_df_origin['POLYLINE'].str.count(',') * 15
starting_point_lon = train_df_origin.apply(lambda x : get_lon(int(x['ORIGIN_STAND'])), axis=1)
starting_point_lat = train_df_origin.apply(lambda x : get_lat(int(x['ORIGIN_STAND'])), axis=1)
features_origin = pd.concat([train_df_origin['HOUR'], train_df_origin['DAY'], train_df_origin['MONTH'], starting_point_lon, starting_point_lat], axis=1)

print(features_nan)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


ValueError: setting an array element with a sequence. The requested array would exceed the maximum number of dimension of 32.

In [ ]:
train, test = df.split_frame(ratios=[0.8], seed = 1)